https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/open-MODIS-hdf4-files-python/


# MOD14A2_analysis

In [14]:
# Import packages
import os
import re  # regular expressions
import warnings
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import rasterio as rio
from rasterio.plot import plotting_extent
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em

warnings.simplefilter('ignore')

# Set working directory
os.chdir(os.path.join(et.io.HOME, 'nyu', 'rbda_project_data'))


In [15]:
# Create a path to the pre-fire MODIS h4 data
thermal_anomalies_path = os.path.join("thermal_anomalies_11_27_test_2018_2019",
                             "MOD14A2.A2018121.h08v04.006.2018129234132.hdf")


In [16]:
# View dataset metadata
with rio.open(thermal_anomalies_path) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta


<open DatasetReader name='thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf' mode='r'>


{'driver': 'HDF4',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [17]:
# Print all of the subdatasets in the data
with rio.open(thermal_anomalies_path) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)


HDF4_EOS:EOS_GRID:thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf:MODIS_Grid_8Day_Fire:FireMask
HDF4_EOS:EOS_GRID:thermal_anomalies_11_27_test_2018_2019/MOD14A2.A2018121.h08v04.006.2018129234132.hdf:MODIS_Grid_8Day_Fire:QA


In [18]:
# Create list to append arrays (of all type of data)
thermal_anomalies_data = []
thermal_anomalies_names = []

# Open the precipitation HDF5 file 
with rio.open(thermal_anomalies_path) as dataset:
    
    # loop through each subdataset in HDF5 file
    for name in dataset.subdatasets:
        
        # Open the subdataset 
        with rio.open(name) as subdataset:
            modis_meta = subdataset.profile
            
            # Read data as a  2 dimensional array and append to list
            thermal_anomalies_data.append(subdataset.read(1))
            thermal_anomalies_names.append(name);
#             np.savetxt(str(name)[-10:-1] + ".csv", subdataset.read(1), delimiter=",")

            
type(thermal_anomalies_data[0])

numpy.ndarray

In [19]:
thermal_anomalies_data[0]

array([[3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 4],
       [3, 3, 3, ..., 4, 4, 3],
       ...,
       [3, 3, 3, ..., 5, 5, 5],
       [3, 3, 3, ..., 5, 5, 5],
       [3, 3, 3, ..., 5, 5, 5]], dtype=uint8)

In [20]:
thermal_anomalies_data[0].shape

(1200, 1200)

In [21]:
thermal_anomalies_data[1]

array([[4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       ...,
       [4, 4, 4, ..., 6, 6, 6],
       [4, 4, 4, ..., 6, 6, 6],
       [4, 4, 4, ..., 6, 6, 6]], dtype=uint8)

In [22]:
thermal_anomalies_data[1].shape

(1200, 1200)